<a href="https://colab.research.google.com/github/luizahenriques/perfilpoliticos/blob/main/Deputados_A%C3%A7%C3%A3odoDeputado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gdown

In [2]:
import urllib3

# Desativar os avisos de InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
import gdown
import pandas as pd
import json

In [4]:
#Obter o arquivo antigo com as ações dos deputados
caminho_do_arquivo = r[PATHDESTINOACAODEPUTADO]
arquivoantigo = pd.read_csv(caminho_do_arquivo)

NameError: name 'r' is not defined

In [ ]:
# Substitua 'ID_DO_ARQUIVO' pelo ID real do arquivo no Google Drive
file_id = [FILEID]

# URL de download direto do Google Drive
url = f'https://drive.google.com/uc?id={file_id}'

# Nome do arquivo de saída
output = 'Noticiasempilhadas.csv'

# Baixar o arquivo
gdown.download(url, output, quiet=False)

In [ ]:
df = pd.read_csv('Noticiasempilhadas.csv')

In [ ]:
# Convertendo a coluna datapublicacao para o formato de data (Necessário para o código seguinte funcionar)
df['datapublicacao'] = pd.to_datetime(df['datapublicacao'])
arquivoantigo['datapublicacao'] = pd.to_datetime(arquivoantigo['datapublicacao'])

In [ ]:
import pandas as pd

# Realize um merge com indicador (_merge) baseado na coluna "datapublicacao"
merged_df = pd.merge(df, arquivoantigo, on='datapublicacao', how='left', indicator=True)

# Mantenha apenas as linhas que estão apenas no dataframe da esquerda (df)
df = merged_df.loc[merged_df['_merge'] == 'left_only', df.columns.difference(['_merge'])]

In [ ]:
#Filtrar o df para não trazer linhas que não tiver deputados encontrados (score relevante)
df = df[df['scorescoluna'] != "[]"]

In [ ]:
num_linhas = df.shape[0]
num_linhas

In [ ]:
df = df.head(2)

In [ ]:
df

In [ ]:
#CRIAR A LISTA DE DEPUTADOS QUE SÃO CITADOS NA NOTÍCIA
import pandas as pd
import re
import math  # Importa o módulo math para verificar NaN

# Função para extrair o segundo elemento de cada tupla, com condições
def extract_second_element(string_data):
    # Verificar se a célula é NaN ou não contém vírgula
    if pd.isna(string_data) or ',' not in string_data:
        return []  # Retorna uma lista vazia se não atender às condições

    matches = re.findall(r"\('(.*?)', '(.*?)', \d+\.\d+\)", string_data)
    return [match[1] for match in matches]

# Aplicar a função à coluna 'scoresdata'
df['deputadoscitados'] = df['scorescoluna'].apply(extract_second_element)

#Criar uma lista única unificando todos os resultados
unique_second_elements = [element for sublist in df['deputadoscitados'] for element in sublist]
# Eliminar itens duplicados

unique_second_elements = list(set(unique_second_elements))


In [ ]:
unique_second_elements

In [ ]:
testelista = unique_second_elements


In [ ]:
import pandas as pd
import requests
import re

# Supondo que você já tenha um DataFrame df com a coluna 'descricao'
# ...

# Função para fazer a requisição e obter as keywords
def obter_acaodeputado(politico,descricao):
    noticia = f'Defina em no máximo 30 tokens qual é a participação de {politico} na notícia a seguir. Se não houver menção a ela no texto, não responda nada. A notícia é: {descricao}'
    url = [URLAPIPETROBRAS]
    headers = {
        'Content-Type': 'application/json',
        'api-key': [APIKEY]
    }
    data = {
        "messages": [
            {
                "role": "system",
                "content": "Você resume notícias e garante que as principais informações sobre o tema serão abordadas"
            },
            {
                "role": "user",
                "content": noticia
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data, verify=False)

    resumo = json.loads(response.text)

    # Verificar se 'content' está presente na resposta antes de retornar
    if 'choices' in resumo and resumo['choices'] and 'message' in resumo['choices'][0] and 'content' in resumo['choices'][0]['message']:
        conteudo = resumo['choices'][0]['message']['content']
        return conteudo
    else:
        # Se 'content' não estiver presente, retornar vazio
        return ''



In [ ]:
#Função para criar a coluna com a Ação do Deputado (obtida via API Open AI) e salvando numa coluna do dataframe
def aplicar_obtencaoacao(row, testelista):
    resultados = {'Politico': [], 'AcaoDeputado': []}

    for p in testelista:
        print(f"Current 'p' value: {p}")
        resumo = obter_acaodeputado(p, row['descricao'])
        resultados['Politico'].append(p)
        resultados['AcaoDeputado'].append(resumo)

    return resultados

df_resultados = df.apply(lambda row: aplicar_obtencaoacao(row, testelista), axis=1, result_type='expand')

df_final = pd.concat([df, df_resultados], axis=1)

In [ ]:
# Criar uma lista de dicionários para cada linha
data = []

for index, row in df_final.iterrows():
    politicos = row['Politico']
    acoes = row['AcaoDeputado']
    for politico, acao in zip(politicos, acoes):
        data.append({'datapublicacao': row['datapublicacao'],
                     'Politico': politico, 'AcaoDeputado': acao})

# Criar um novo DataFrame a partir da lista de dicionários
df_expandido = pd.DataFrame(data)

In [ ]:
arquivoantigo['datapublicacao'] = pd.to_datetime(arquivoantigo['datapublicacao'])
df_expandido['datapublicacao'] = pd.to_datetime(df_expandido['datapublicacao'])

df_concatenado = pd.concat([arquivoantigo, df_expandido], ignore_index=True)

In [ ]:
df_concatenado

In [ ]:
temp_csv_path = r[PATHDESTINOACAODEPUTADO]
df_concatenado.to_csv(temp_csv_path,mode='w',index=False)